# Time Machine Surf Cam

This is a primitive intial version and simply implements a nearest neighbour approach on key features which have been chosen some what arbitrarily (informed decision) and have all been given equal weight.

We first load the data in from the database.

## Data Loading

In this notebook, we are working off a duplicate of the 04/10/23 version of the [surf-forecast-videos](https://github.com/jreaso/surf-forecast-videos) dataset.